In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

# Display the version
print(tf.__version__)

# import and mount Drive
import os
from google.colab import drive
drive.mount('/content/gdrive')

2.13.0
Mounted at /content/gdrive


In [2]:
# get path to folder file is in
!pip install kora
from kora.drive import get_path
file_path = get_path('17rNCLBHSPszyoaacFSpsFGD5nlP1OBF7')
file_path = '/content/gdrive/' + str(file_path) + '/CIFAR-100'
print(file_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
/content/gdrive/My Drive/Personal Projects/Pioneer Academics Research/code/CIFAR-100


In [3]:
# Download the data from tf, unless it's already here.
if not os.path.exists(file_path+'/data/clean'):
  os.mkdir(file_path+'/data/clean')
  (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
  np.save(file_path+'/data/clean/X_train.npy',X_train)
  np.save(file_path+'/data/clean/y_train.npy',y_train)
  np.save(file_path+'/data/clean/X_test.npy',X_test)
  np.save(file_path+'/data/clean/y_test.npy',y_test)
else:
  X_train = np.load(file_path+'/data/clean/X_train.npy')
  y_train = np.load(file_path+'/data/clean/y_train.npy')
  X_test = np.load(file_path+'/data/clean/X_test.npy')
  y_test = np.load(file_path+'/data/clean/y_test.npy')

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
X_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [4]:
# Converting to float32 and reducing pixel values
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

In [5]:
# Code to train a model
def train(params=[64, 64, 128, 128, 256, 256], labels=y_train, file_name=(file_path + '/models/defensive_distillation.keras'), temperature = 20, num_epochs = 50, lr = 0.0003, drop = 0.2, reg = 1e-4, batch_size=32):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(params[0], (3, 3), input_shape=X_train.shape[1:], padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(reg)))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Dropout(drop))

    model.add(tf.keras.layers.Conv2D(params[1], (3, 3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(reg)))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Conv2D(params[2], (3, 3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(reg)))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Conv2D(params[3], (3, 3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(reg)))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Dropout(drop))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(params[4], activation='relu', kernel_regularizer=tf.keras.regularizers.L2(reg)))
    model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(params[5], activation='relu', kernel_regularizer=tf.keras.regularizers.L2(reg)))
    model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(100, kernel_regularizer=tf.keras.regularizers.L2(reg)))

    def temperature_cross_entropy(gt, pred):
      loss = tf.nn.softmax_cross_entropy_with_logits(labels=gt, logits=pred/temperature)
      return loss

    model.compile(loss=temperature_cross_entropy,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['accuracy'])

    model.summary()

    model.fit(X_train, labels,
              batch_size=batch_size,
              validation_split=0.1,
              epochs=num_epochs,
              shuffle=True,
              verbose=1)

    if file_name != None:
        model.save(file_name)

    return model

In [6]:
# Create the teacher model
model = train(file_name=(file_path + '/models/distillation_teacher.keras'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 8, 8, 64)          0

In [7]:
# Evaluate efficacy of teacher model
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.3801 - accuracy: 0.4473


[2.3800530433654785, 0.447299987077713]

In [8]:
# Create 'soft labels' for Defensive Distillation
y_soft_train = model.predict(X_train, batch_size=32)
y_soft_train = tf.nn.softmax(y_soft_train/20)
print(y_soft_train.shape)

1563/1563 [==============================] - 3s 2ms/step
(50000, 100)


In [9]:
# Create the student (distilled) model
distilled_model = train(labels=y_soft_train, file_name=(file_path + '/models/distillation_student.keras'))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 8, 8, 64)         

In [10]:
# Evaluate efficacy of student (distilled) model on a set the model has never seen
distilled_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.4262 - accuracy: 0.4088


[2.4262473583221436, 0.40880000591278076]